## API Versioning with Tyk
This sample is intended to show how a service can migrate from one version to the next and be proxied through Tyk. The service will have multiple versions, and Tyk will also have multiple versions. Tyk represents what will become our new public API. The service in this case is an HTTP version of the [Jokes Service](https://github.com/datarobot/jokes-service/). 

There will be three different versions of the service that are run:
- v1.0 has one operation: GET /tell/joke.
- v2.0 adds an operation: POST /tell/joke

These will be fronted by Tyk, where it will accept a version header. The first version will be v5.8.0, then the next will be v6.0.0. These correspond with the different versions of the backing HTTP service (v1 and 2). Both APIs will be active at one time, but have some differences that make them incompatible.

In [6]:
# ATTENTION: this is all just setup code. None of it really matters in terms of what we are doing with Tyk, with the exception of the `define_api` function.
import docker
import kubernetes
import requests
import os
import subprocess
import time

session = requests.Session()
session.headers.update({'x-tyk-authorization': 'stuff'})

tyk_namespace = 'tyk'
service_namespace = 'http-jokes'

def cleanup_apis(requests_session):
    for api in requests_session.get("http://localhost:8080/tyk/apis/").json():
        print(api['api_id'])
        print(requests_session.delete("http://localhost:8080/tyk/apis/" + api['api_id']).text)
    requests_session.get('http://localhost:8080/tyk/reload/group').text


def define_api(openapi_file, requests_session, api_id=None):
    with open(openapi_file, 'r') as f:
        api_definition = f.read()
    if api_id:
        response = requests_session.put(f"http://localhost:8080/tyk/apis/oas/{api_id}", data=api_definition)
    else:
        response = requests_session.post("http://localhost:8080/tyk/apis/oas", data=api_definition)
    out = response.json()
    display(out)
    display(requests_session.get('http://localhost:8080/tyk/reload/group').text)
    return out.get('key')

def install_tyk():
    client, config = kubernetes.client, kubernetes.config
    config.load_kube_config()
    k8s = client.CoreV1Api()

    create_k8s_namespace(client, k8s, tyk_namespace)

    try:
        k8s.read_namespaced_config_map('tyk-secret', namespace=tyk_namespace)
    except kubernetes.client.rest.ApiException:
        cmap = client.V1ConfigMap()
        cmap.metadata = client.V1ObjectMeta(name='tyk-secret')
        cmap.data = {'APISecret': 'stuff'}
        k8s.create_namespaced_config_map(namespace=tyk_namespace, body=cmap)

    os.system(f'helm upgrade tyk-redis oci://registry-1.docker.io/bitnamicharts/redis -n {tyk_namespace} --install --version 19.0.2 --set replica.replicaCount=1 --wait')
    os.system(f'helm upgrade tyk-oss tyk-helm/tyk-oss -n {tyk_namespace} --install -f tyk-values.yaml --wait')


def setup_service():
    # first build the image, then run it in k8s
    dckr = docker.from_env()
    image, _ = dckr.images.build(path='.', tag=f'http-jokes:2.0', rm=True)
    image.tag(f'localhost:5001/http-jokes:latest')
    dckr.images.push(f'localhost:5001/http-jokes:latest')

    client, config = kubernetes.client, kubernetes.config
    config.load_kube_config()
    k8s = client.CoreV1Api()
    create_k8s_namespace(client, k8s, service_namespace)
    os.system(f'kubectl -n {service_namespace} apply -f app-deploy.yaml')


def create_k8s_namespace(client, k8s, namespace):
    try:
        ns = k8s.read_namespace(namespace)
    except kubernetes.client.rest.ApiException:
        # create namespace
        ns = client.V1Namespace(metadata=client.V1ObjectMeta(name=namespace))
        k8s.create_namespace(ns)
    return ns


def kill_namespace(namespace):
    client, config = kubernetes.client, kubernetes.config
    config.load_kube_config()
    k8s = client.CoreV1Api()
    try:
        k8s.delete_namespace(namespace, grace_period_seconds=0)
    except kubernetes.client.rest.ApiException:
        pass

### Running Tyk, Building the Service
The process starts with installing and running Tyk. Uncomment the `install_tyk()` line if you do not have it running. 

This uses the 'hello' endpoint in Tyk to make sure that it is running. If you do not see a line like the following, then something is not configured properly.

```
{'status': 'pass', 'version': '5.5.0', 'description': 'Tyk GW', 'details': {'redis': {'status': 'pass', 'componentType': 'datastore', 'time': '2024-09-26T19:57:37Z'}}}
```

In [7]:
# if needed, run this cell to get tyk up and running
#install_tyk()
setup_service()

# would be nice to check if this is alreay running from another place somehow.
port_fwd = subprocess.Popen(['kubectl', '-n', tyk_namespace, 'port-forward', 'svc/gateway-svc-tyk-oss-tyk-gateway', '8080:8080'])
time.sleep(1)
display(session.get('http://localhost:8080/hello').json())

service/http-jokes created
deployment.apps/http-jokes-service created
Forwarding from 127.0.0.1:8080 -> 8080
Forwarding from [::1]:8080 -> 8080
Handling connection for 8080


{'status': 'pass',
 'version': '5.5.0',
 'description': 'Tyk GW',
 'details': {'redis': {'status': 'pass',
   'componentType': 'datastore',
   'time': '2024-09-27T18:12:21Z'}}}

### Version 1.0
The next block builds a docker image of the Jokes service code in `api/main.py` and then pushes it to a local registry. It then deploys that container to your k8s instance and registers the service with Tyk. That will generate some output that looks like:
```
{'key': 'ba1b323912a8468b90a69811c937afcf', 'status': 'ok', 'action': 'added'}
```
Finally, we make a request to Tyk that is proxied to the service. The final output from the cell will be a response from the Jokes service. That should look like the following:
```
{'value': 'Hey, funny thing... this HTTP stuff does work...'}
```

In [10]:
cleanup_apis(session)
base_api_id = define_api('openapi-v1.json', session)
time.sleep(1)
# get proxied joke
display(requests.get('http://localhost:8080/jokes/tell/joke', headers={ 'x-api-version': 'v5.8.0'}).json())

2bcde6bf32f8460f8587098eb719fadf
{"key":"2bcde6bf32f8460f8587098eb719fadf","status":"ok","action":"deleted"}

47c9e295f1d247fba4c53d76c71eae1d
{"key":"47c9e295f1d247fba4c53d76c71eae1d","status":"ok","action":"deleted"}



{'key': 'e4a128ab59374988a057fa3e5c34ffec', 'status': 'ok', 'action': 'added'}

'{"status":"ok","message":""}\n'

Handling connection for 8080


{'value': 'Hey, funny thing... this HTTP stuff does work...'}

### Version 2.0 
Now we deploy the next version of the service, which is a major version bump to 6.0 for the external API, and 2.0 for the internal API. It adds a way to get the joke with a POST request, and we pass in a `joke_type` to get different output from the service. It should display the same output as the previous instance. There is also a request with to the GET endpoint to illustrate that it also runs with the version 1.1 of the service. The final request is a POST but using v5.8, which should prevent that operation from running.

In [11]:
def version_api(base_api_id, openapi_file, requests_session):
    with open(openapi_file, 'r') as f:
        api_definition = f.read()
    response = requests_session.post("http://localhost:8080/tyk/apis/oas", data=api_definition, params={'base_api_id': base_api_id, 'base_api_version_name': 'v5.8.0', 'new_version_name': 'v6.0.0', 'set_default': 'false'})
    response.raise_for_status()
    child_api_id = response.json().get('key')
    display(child_api_id)
    display(requests_session.get('http://localhost:8080/tyk/reload/group').text)
    return child_api_id

v1_1_api_id = version_api(base_api_id, 'openapi-v2.0.json', session)
time.sleep(1)

# post to get proxied joke (terrible pattern, but just a demo)
display(requests.post('http://localhost:8080/jokes/tell/joke', json={'joke_type': 2}, headers={'x-api-version': 'v6.0.0'},).json(), clear=True)
display(requests.get('http://localhost:8080/jokes/tell/joke', headers={ 'x-api-version': 'v6.0.0'}).json())

# when trying to POST to the old version, we get Method Not Allowed
display(requests.post('http://localhost:8080/jokes/tell/joke', json={'joke_type': 2}, headers={'x-api-version': 'v5.8.0'},).json())


{'value': "Who's there? Must be a service_registry user?"}

Handling connection for 8080


{'value': 'Hey, funny thing... this HTTP stuff does work...'}

Handling connection for 8080


{'detail': 'Method Not Allowed'}

### Cleanup

The last cell just cleans things up. If you also want to cleanup Tyk, uncomment the last two lines.

In [5]:
kill_namespace(service_namespace)
cleanup_apis(session)
port_fwd.kill()
# os.system(f'helm uninstall -n {tyk_namespace} tyk-oss')
# kill_namespace(tyk_namespace)

df08c1da15a340eaa823227e92d8b5be
{"key":"df08c1da15a340eaa823227e92d8b5be","status":"ok","action":"deleted"}

0c89b52b95404436b4e3eff1f9700092
{"key":"0c89b52b95404436b4e3eff1f9700092","status":"ok","action":"deleted"}

